In [1]:
# Import all required libraries
import pandas as pd
from pandas import *
import numpy
from datetime import datetime

# Initialize values

env = None
sample_size = None
# Set env, if env = test, will only be run locally and display the result
env = "prod"
env = "test"

nb_models = 1

# Number of value on which to train, if null, train on all value
sample_size = None
test_size = 1000

In [2]:
# Read training data + test data
df_data_input = pandas.read_json("../input/train.json")
df_test_input = pandas.read_json("../input/test.json")
df_sample_input = pandas.read_csv("../input/sample_submission.csv")

print("df_data contais {0} elements".format(df_data_input.count()))
# Display basic information
display(df_data_input.head(3))
display(df_test_input.head(3))
display(df_sample_input.head(2))

df_data contais cuisine        39774
id             39774
ingredients    39774
dtype: int64 elements


,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."


,id,ingredients
0,18009,"[baking powder, eggs, all-purpose flour, raisi..."
1,28583,"[sugar, egg yolks, corn starch, cream of tarta..."
2,41580,"[sausage links, fennel bulb, fronds, olive oil..."


,id,cuisine
0,35203,italian
1,17600,italian


In [3]:
if env == "test":
    if sample_size is not None and sample_size > 0:
        df_data = df_data_input.sample(sample_size)
    else:
        df_data = df_data_input
    df_test = df_data_input.sample(test_size)
    
    # Removing all df_test from df_data to ensure not train with test data
    df_common = df_data.merge(df_test,on=['id'])
    #display(df_common)
    df_data = df_data[(~df_data.id.isin(df_common.id))]
else:
    # set that to some default value
    df_test = df_test_input
    df_test['cuisine'] = "todo"
    if sample_size is not None and sample_size > 0:
        df_data = df_data_input.sample(sample_size)
    else:
        df_data = df_data_input
        
print("df_data contais {0} elements".format(df_data.count()))
# Display basic information
display(df_data.head(3))
display(df_test.head(3))

df_data contais cuisine        38774
id             38774
ingredients    38774
dtype: int64 elements


,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."


,cuisine,id,ingredients
22391,moroccan,11908,"[water, dried apricot, cinnamon sticks, sliver..."
6760,italian,25498,"[green bell pepper, garlic, onions, zucchini, ..."
26816,mexican,31417,"[green bell pepper, olive oil, cilantro, onion..."


In [4]:
# Extract columns values of ingredients to multiple columns with boolean
import re
def preprocess_dataframe(df1, df2):
    count1 = len(df1)
    count2 = len(df2)
    
    print("count1: {0}".format(count1))
    print("count2: {0}".format(count2))
    
    total_df = df1.append(df2, ignore_index=True)
    
    # Remove values from ingredients
    print("Removing wrong values")
    #total_df['ingredients'] = total_df['ingredients'].map(lambda x: re.sub(r'[0-9]+', '', x))
    print("end removing wrong values")
    d_list = []
    for index, row in total_df.iterrows():
        for value in row['ingredients']:
            #print(value)
            new_value = re.sub(r'[0-9]+', '', value)
            new_value = new_value.replace('s', '')

            d_list.append({'id':row['id'], 
                           'value':new_value})
    total_df = total_df.append(d_list, ignore_index=True)
    total_df = total_df.groupby('id')['value'].value_counts()
    total_df = total_df.unstack(level=-1).fillna(0)
    
    # Remove all values present less than 5 times
    
    
    # Then, we need to merge df_1 and df_2 with their id
    df1 = df1.merge(total_df, left_on='id', right_on='id', how='inner')
    df2 = df2.merge(total_df, left_on='id', right_on='id', how='inner')

    # We do not need the ingredients column now, so, we can remove it
    df1 = df1.drop(columns=['ingredients'])
    df2 = df2.drop(columns=['ingredients'])

    return df1, df2

df_data_1, df_test_1 = preprocess_dataframe(df_data, df_test)



def category_to_number(df):
    df["code_cuisine"] = df.cuisine.astype("category").cat.codes
    mapping = df[["cuisine", "code_cuisine"]]
    return df, mapping.drop_duplicates()

#df_data_1, code_mapping = category_to_number(df_data_1)
#df_test_1, empty = category_to_number(df_test_1)


display(df_data.head(5))
display(df_data_1.head(5))


count1: 38774
count2: 1000
Removing wrong values
end removing wrong values


/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


,cuisine,id,"/ to lb. chicken, cut into erving piece",Up,"ounc ziti pata, cook and drain",% le odium chicken broth,% le odium cooked deli ham,% le odium cooked ham,% le odium ham,% le odium moked fully cooked ham,...,yuzu juice,za'atar,zechuan auce,zechwan peppercorn,zet,zety italian dreing,zinfandel,ziti,zucchini,zucchini bloom
0,greek,10259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,southern_us,25693,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,filipino,20130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,indian,22213,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,indian,13162,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# Generate our training/validation datasets
from sklearn import model_selection

# Name of the result column
result_cols = ['cuisine']
result_excl_cols = ['cuisine_']

input_cols = [
    ''
]
input_excl_cols = ['ingredients', 'cuisine', 'code_cuisine']
# Removing input_cols = ['store', 'item',
# dom, cw, 

# Train on everything

# Get the final values
def get_values(df, cols=[], excl_cols = []):
    columns = df.columns.values
    # Remove all columns that are not inside the list
    cols_to_drop = []
    for column in columns:
        find = False
        ignore = False
        for excl_col in excl_cols:
            if column.startswith(excl_col):
                ignore = True
        if ignore is False:
            for col in cols:
                if column.startswith(col):
                    find = True
        if not find:
            cols_to_drop.append(column)
    print("dropping columns")
    df = df.drop(columns=cols_to_drop)
    print("end dropping columns")
    new_order = sorted(df.columns.values)
    # Same order for both training and testing set
    df = df[new_order]
    return df.values

X_train = get_values(df_data_1, input_cols, input_excl_cols)
X_test = get_values(df_test_1, input_cols, input_excl_cols)

Y_train = get_values(df_data_1, result_cols, result_excl_cols).ravel()

# In test env, we calculate it for the test only
if env == "test":
    Y_test = get_values(df_test_1, result_cols, result_excl_cols).ravel()

dropping columns
end dropping columns
dropping columns
end dropping columns
dropping columns
end dropping columns
dropping columns
end dropping columns


In [6]:
# Normalize the data
X_all = [x + y for x, y in zip(X_train, X_test)]

from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler() 

# Don't cheat - fit only on training data
# Def adding x_train + X_test + X_validation to fit all of them
scaler.fit(X_train )  

X_train = scaler.transform(X_train) 
X_test = scaler.transform(X_test) 

In [7]:
# Custom function to calculate the score
def get_score(Y_validation, Y_validation_predict):
    nb_success = 0
    for i in range(0, len(Y_validation)):
        #print("value1 {0} = {1}".format(Y_validation[i], Y_validation_predict[i]))
        if Y_validation[i] == Y_validation_predict[i]:
            nb_success += 1
    return nb_success / len(Y_validation) * 100

In [8]:
# Import algorithm
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import *
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

import lightgbm as lgbm

models = []

#models.append(('LogisticRegression', LogisticRegression()))
#models.append(('KNeighborsClassifier', KNeighborsClassifier()))
#models.append(('LinearDiscriminantAnalysis', LinearDiscriminantAnalysis()))
#models.append(('GaussianNB', GaussianNB()))
#models.append(('SVC', SVC()))

for i in range(5, 5 + nb_models):
    
        # training with early stop
    models.append(('lgbm_{0}'.format(i), 
                   lgbm.sklearn.LGBMClassifier(boosting_type= 'gbdt',
                                               random_state=i*10000,
          #objective = 'binary',
          #silent = True,
          #max_depth = -1,
          #min_data_in_leaf=1,
          #max_bin = 512,
          #subsample_for_bin = 200,
          #subsample = 1,
          #subsample_freq = 1,
          #min_split_gain = 0.5,
          #min_child_weight = 1,
          #min_child_samples = 5,
          #scale_pos_weight = 1,
           # Updated from 'nthread'
          n_jobs = 7 )
                  ))
# High value until first model get solved
best_model = "UNKNOWN"

res = []
# Testing all models, one by one
for name, model in models:
    print("Executing for model {0}".format(name))
    time_start = datetime.now()

    # Training the model
    model.fit(X_train, Y_train)
    print("Finish fit for {0}".format(name))

    Y_test_result = model.predict(X_test)
    res.append(Y_test_result)
    if env == "test":
        # We can calculate the avg error
        score = get_score(Y_test, Y_test_result)
        print("Model {0} got score of {1}, time: {2}".format(name, score, datetime.now() - time_start))
    else:
        # Let's write an output file, with the name of the model
        print("Writing output file {0}.csv for model {0}".format(name))
        
        df_test['cuisine'] = Y_test_result
        result_df = df_test[['id', 'cuisine']]
        result_df['cuisine'] = Y_test_result
        
        result_df.to_csv("{0}.csv".format(name), index=False)

Executing for model lgbm_5
Finish fit for lgbm_5
Model lgbm_5 got score of 76.8, time: 0:03:18.114562


In [9]:
# For all result in res, if test, display the result, if not, write it to a file
final_res = []
nb_variable = len(res[0])
for variable in range(0, nb_variable):
    final_res.append(0.0)
    dict_cuisine = {}
    for i in range(0, len(res)):
        cuisine_found = res[i][variable]
        if cuisine_found in dict_cuisine:
            dict_cuisine[cuisine_found] += 1
        else:
            dict_cuisine[cuisine_found] = 1
    # Now, we need to find the most common one for all the values inside dict_cuisine
    current_value = 0
    current_cuisine = ""
    for cuisine in dict_cuisine:
        if dict_cuisine[cuisine] > current_value:
            current_cuisine = cuisine
            current_value = dict_cuisine[cuisine]
    
    final_res[variable] = current_cuisine

if env == "test":
    # We can calculate the avg error
    score = get_score(Y_test, final_res)
    print("avg model got score of {0}".format(score))
else:
    print("Writing output file merged.csv".format(name))

    df_test['cuisine'] = final_res
    result_df = df_test[['id', 'cuisine']]
    result_df['cuisine'] = final_res

    result_df.to_csv("merged.csv".format(name), index=False)

avg model got score of 76.8
